# G2Product Guide - Version

The `version()` method creates a JSON string with information about your Senzing version.

More information:

1. [GitHub repository](https://github.com/Senzing/docker-jupyter)
1. [Senzing documentation](http://docs.senzing.com/?python#g2config)

## Prepare environment

In [ ]:
import com.senzing.g2.engine.G2Product;
import com.senzing.g2.engine.G2ProductJNI;

### Helper class for Json Rendering

In [ ]:
%%loadFromPOM
<dependency>
    <groupId>org.glassfish</groupId>
    <artifactId>javax.json</artifactId>
    <version>1.1.4</version>
</dependency>

In [ ]:
import javax.json.*;
import static java.util.Collections.*;
import static javax.json.stream.JsonGenerator.PRETTY_PRINTING;

In [ ]:
public class JsonUtil {
    private static final JsonWriterFactory PRETTY_FACTORY
        = Json.createWriterFactory(singletonMap(PRETTY_PRINTING, true));
        
    private static final JsonWriterFactory UGLY_FACTORY
        = Json.createWriterFactory(emptyMap());
    
    public static String toJsonText(JsonValue val) {
        return toJsonText(val, true);
    }

    public static String toJsonText(JsonValue val, boolean prettyPrint) {
        JsonWriterFactory factory = (prettyPrint) ? PRETTY_FACTORY : UGLY_FACTORY;
        StringWriter sw = new StringWriter();
        JsonWriter writer = factory.createWriter(sw);
        writer.write(val);
        sw.flush();
        return sw.toString();
    }
    
    public static JsonObject parseJsonObject(String jsonText) {
        if (jsonText == null) return null;
        StringReader sr = new StringReader(jsonText);
        JsonReader jsonReader = Json.createReader(sr);
        return jsonReader.readObject();
  }

    public static JsonArray parseJsonArray(String jsonText) {
        if (jsonText == null) return null;
        StringReader sr = new StringReader(jsonText);
        JsonReader jsonReader = Json.createReader(sr);
        return jsonReader.readArray();
  }
    
    
}

In [ ]:
import java.util.UUID;
public static void RenderJSON(String str){
    JsonObject json = JsonUtil.parseJsonObject(str);
    String Config = JsonUtil.toJsonText(json, false);
    UUID id = UUID.randomUUID();
    String uuid = id.toString();
    String div = "<div id=\""+ uuid +"\" style=\"height:100%; width:100%; background-color: LightCyan\"></div>";
    display(div, "text/html");
    String jav = "require([\"https://rawgit.com/caldwell/renderjson/master/renderjson.js\"], function() {document.getElementById(\'"+ uuid +"\').appendChild(renderjson("+json+"))});";
    display(jav, "application/javascript");
}

### Initialize Senzing configuration

Using environment variables and default values, create `senzingConfigJson`.
This value is used when instantiating Senzing objects.

In [ ]:
// Get variables used in constructing Senzing Engine configuration.

String configPath = System.getenv("SENZING_ETC_DIR");
if (configPath == null) {
    configPath = "/etc/opt/senzing";
}

String supportPath = System.getenv("SENZING_DATA_VERSION_DIR");
if (supportPath == null) {
    supportPath = "/opt/senzing/data";
}

String g2Path = System.getenv("SENZING_G2_DIR");
if (g2Path == null) {
    g2Path = "/opt/senzing/g2";
}

String resourcePath = g2Path + "/resources";

String sqlConnection = System.getenv("SENZING_SQL_CONNECTION");
if (sqlConnection == null) {
    sqlConnection = "sqlite3://na:na@/var/opt/senzing/sqlite/G2C.db";
}

// Construct the JSON string used for Senzing Engine configuration.

String senzingConfigJson = "{"
   + "\"PIPELINE\": {"
   +     "\"CONFIGPATH\": \"" + configPath + "\","
   +     "\"SUPPORTPATH\": \"" + supportPath + "\","
   +     "\"RESOURCEPATH\": \"" + resourcePath + "\""
   + "},"
   + "\"SQL\": {"
   +     "\"CONNECTION\": \"" + sqlConnection + "\""
   + "}}";
   
System.out.println(senzingConfigJson);

## G2Product

### Create G2Product instance

In [ ]:
G2Product g2product = new G2ProductJNI();

String moduleName = "ExampleG2Config";
boolean verboseLogging = true;
int returnCode = g2product.initV2(moduleName, senzingConfigJson, verboseLogging);
System.out.print(returnCode);

### version()

Call G2Product's `version()` method and print the results.

In [ ]:
String version_string = g2product.version();

System.out.print(version_string);